# Доля аудитории по баинговым группам в 100+
Пример расчета Доли телеканалов по баинговым аудиториям.

## Описание задачи и условий расчета
- Период: 01.08.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 18+ 100K+, Все 14-44 100K+, Все 11-34 100K+, Все 14-59 100K+, Женщины 14-39 100K+, Женщины 25-59 100K+, Все 25-59 100K+, Мужчины  25-59 100K+, Все 10-45 100K+, Все 25+ 100K+, Все 25-49 100K+, Все 29+ 100K+, Все 4-45 100K+, Все 25-54 100K+, Мужчины  25+ 100K+, Все 25-55 100K+, Все 20-44 100K+, Мужчины  18-45 100K+, Женщины 20-45 100K+, Все 18-44 100K+, Мужчины  18+ 100K+, Женщины 14-44 100K+, Мужчины  14-59 100K+, Все 4-40 100K+, Мужчины  25-49 100K+, Женщины 25-49 100K+, Женщины 18-45 100K+, Все 18-59 100K+
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ, РОССИЯ 1, НТВ, ПЯТЫЙ КАНАЛ, ТНТ 4									
- Статистики: Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [7]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [8]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ", "РОССИЯ 1", "НТВ", "ПЯТЫЙ КАНАЛ", "ТНТ 4"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [9]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','возраст','100+'])

Запрошены записи: 0 - 131
Всего найдено записей: 131



,id,valueId,name,colName,valueName
0,1,1,Пол,Sex,Мужчины
1,1,2,Пол,Sex,Женщины
2,26,1,Семейное положение,MaritalStatus,Женат / замужем
3,26,2,Семейное положение,MaritalStatus,Холост / не замужем
4,2,33,Возраст,Age,33
...,...,...,...,...,...
126,2,10,Возраст,Age,10
127,2,89,Возраст,Age,89
128,2,8,Возраст,Age,8
129,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [10]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (1, 2, 4, 259, 329)'

# Указываем список статистик для расчета
statistics = ['Share']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [11]:
# Задаем необходимые группы
targets = {
    'Все 18+ 100K+':'age >= 18 AND cube100Plus100Minus = 1',
    'Все 14-44 100K+':'age >= 14 AND age <= 44 AND cube100Plus100Minus = 1',
    'Все 11-34 100K+':'age >= 11 AND age <= 34 AND cube100Plus100Minus = 1',
    'Все 14-59 100K+':'age >= 14 AND age <= 59 AND cube100Plus100Minus = 1',
    'Женщины  14-39 100K+':'age >= 14 AND age <= 39 AND sex = 2 AND cube100Plus100Minus = 1',
    'Женщины  25-59 100K+':'age >= 25 AND age <= 59 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 25-59 100K+':'age >= 25 AND age <= 59 AND cube100Plus100Minus = 1',
    'Мужчины 25-59 100K+':'age >= 25 AND age <= 59 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 10-45 100K+':'age >= 10 AND age <= 45 AND cube100Plus100Minus = 1',
    'Все 25+ 100K+':'age >= 25 AND cube100Plus100Minus = 1',
    'Все 25-49 100K+':'age >= 25 AND age <= 49 AND cube100Plus100Minus = 1',
    'Все 29+ 100K+':'age >= 29 AND cube100Plus100Minus = 1',
    'Все 4-45 100K+':'age >= 4 AND age <= 45 AND cube100Plus100Minus = 1',
    'Все 25-54 100K+':'age >= 25 AND age <= 54 AND cube100Plus100Minus = 1',
    'Мужчины 25+ 100K+':'age >= 25 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 25-55 100K+':'age >= 25 AND age <= 55 AND cube100Plus100Minus = 1',
    'Все 20-44 100K+':'age >= 20 AND age <= 44 AND cube100Plus100Minus = 1',
    'Мужчины 18-45 100K+':'age >= 18 AND age <= 45 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  20-45 100K+':'age >= 20 AND age <= 45 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 18-44 100K+':'age >= 18 AND age <= 44 AND cube100Plus100Minus = 1',
    'Мужчины 18+ 100K+':'age >= 18 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  14-44 100K+':'age >= 14 AND age <= 44 AND sex = 2 AND cube100Plus100Minus = 1',
    'Мужчины 14-59 100K+':'age >= 14 AND age <= 59 AND sex = 1 AND cube100Plus100Minus = 1',
    'Все 4-40 100K+':'age >= 4 AND age <= 40 AND cube100Plus100Minus = 1',
    'Мужчины 25-49 100K+':'age >= 25 AND age <= 49 AND sex = 1 AND cube100Plus100Minus = 1',
    'Женщины  25-49 100K+':'age >= 25 AND age <= 49 AND sex = 2 AND cube100Plus100Minus = 1',
    'Женщины  18-45 100K+':'age >= 18 AND age <= 45 AND sex = 2 AND cube100Plus100Minus = 1',
    'Все 18-59 100K+':'age >= 18 AND age <= 59 AND cube100Plus100Minus = 1'
}

## Расчет задания

In [12]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Отправляем задания на расчет
............................
Ждем выполнения
Расчет задач (28) [ = = = = = = = = = = = = ] время расчета: 0:01:22.970471
Расчет завершен, получаем результат
Собираем таблицу
............................

,prj_name,tvNetName,Share
0,Все 18+ 100K+,РОССИЯ 1,16.471973
1,Все 18+ 100K+,ПЕРВЫЙ КАНАЛ,9.279370
2,Все 18+ 100K+,ПЯТЫЙ КАНАЛ,7.075034
3,Все 18+ 100K+,ТНТ 4,0.509303
4,Все 18+ 100K+,НТВ,9.320011
...,...,...,...
0,Все 18-59 100K+,РОССИЯ 1,10.664214
1,Все 18-59 100K+,ПЕРВЫЙ КАНАЛ,7.629500
2,Все 18-59 100K+,ПЯТЫЙ КАНАЛ,5.615798
3,Все 18-59 100K+,ТНТ 4,0.823403


Настраиваем внешний вид таблицы

In [13]:
df_res = pd.pivot_table(df, values=['Share'],
                        index=['prj_name'], 
                        columns=['tvNetName'])
df_res

Share                                              
tvNetName                  НТВ ПЕРВЫЙ КАНАЛ ПЯТЫЙ КАНАЛ   РОССИЯ 1     ТНТ 4
prj_name                                                                    
Все 10-45 100K+       4.900954     5.724491    3.998853   6.639596  1.081657
Все 11-34 100K+       4.025167     4.924617    3.517397   5.925038  0.795362
Все 14-44 100K+       4.804560     5.746419    4.001724   6.556986  1.079529
Все 14-59 100K+       6.829342     7.581290    5.591978  10.561632  0.818695
Все 18+ 100K+         9.320011     9.279370    7.075034  16.471973  0.509303
Все 18-44 100K+       4.843630     5.767300    3.984065   6.597311  1.099576
Все 18-59 100K+       6.890592     7.629500    5.615798  10.664214  0.823403
Все 20-44 100K+       4.841684     5.822754    3.989026   6.584414  1.104680
Все 25+ 100K+         9.431644     9.370815    7.147188  16.673231  0.485695
Все 25-49 100K+       5.892368     6.598703    4.570941   7.872280  1.022612
Все 25-54 100K+       6.330178     7.050604    5.123552   9.168641  0.917935
Все 25-55 100K+       6.402266     7.270076    5.253083   9.507839  0.892777
Все 25-59 100K+       6.996362     7.730926    5.688987  10.792854  0.792359
Все 29+ 100K+         9.531025     9.488101    7.249965  16.928979  0.475366
Все 4-40 100K+        4.096557     4.620082    3.444208   5.622157  0.841200
Все 4-45 100K+        4.547207     5.283927    3.797929   6.103011  0.976974
Женщины  14-39 100K+  4.049009     5.023324    3.942366   5.313362  0.658943
Женщины  14-44 100K+  4.413476     5.575251    4.286862   5.669373  0.773214
Женщины  18-45 100K+  4.727777     5.749113    4.479140   5.912700  0.767558
Женщины  20-45 100K+  4.692673     5.806717    4.485250   5.843289  0.789309
Женщины  25-49 100K+  5.650701     6.658956    5.264400   7.236003  0.648467
Женщины  25-59 100K+  6.891997     7.867888    6.539473  10.242265  0.509496
Мужчины 14-59 100K+   6.966756     7.504446    4.704800  11.249129  1.159137
Мужчины 18+ 100K+     9.243715     9.032294    5.449642  15.442493  0.844598
Мужчины 18-45 100K+   5.483409     6.133506    3.579076   7.768863  1.581953
Мужчины 25+ 100K+     9.327272     9.067351    5.469303  15.577176  0.803604
Мужчины 25-49 100K+   6.163187     6.531183    3.793828   8.585312  1.441891
Мужчины 25-59 100K+   7.118637     7.570460    4.692548  11.437931  1.123765

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [14]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_share_buying_targets'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()